# Resample an Imbalanced Dataset
Resampling is an important technique when you have a classification problem with an imbalance in classes. An imbalanced dataset can cause machine learning models to perform poorly and overpredict the majority class. In some settings, this type of bias can result in major costs (such as fraud detection and medical diagnosis). Balancing a dataset can be a way to avoid the issues that arise from a class imbalance and ensure that your classification model performs well.

You can use this template to resample a dataset of your own based on your needs.

The cell below imports the necessary libraries for this template, configures visualization settings, and defines a function to help interpret the outcomes of different resampling methods.

In [ ]:
# Data manipulation and visualization imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting size and style
sns.set(rc = {'figure.figsize':(15, 8)})
sns.set_style("darkgrid")

# Preprocessing imports
from sklearn.model_selection import train_test_split

# Create a function to plot resampled data
def plot_resample(X, y, X_res, y_res, method):    
    # Create subplots
    fig, axes = plt.subplot_mosaic([['a', 'b'], ['c', 'd']])
	# Generate scatterplots of first two features for inspection
    sns.scatterplot(x=X.iloc[y.values == 0, 0], y=X.iloc[y.values == 0, 1], alpha=0.10, ax=axes['a'])
    sns.scatterplot(x=X.iloc[y.values == 1, 0], y=X.iloc[y.values == 1, 1], alpha=0.10, ax=axes['a'])
    sns.scatterplot(x=X_res.iloc[y_res.values == 0, 0], y=X_res.iloc[y_res.values == 0, 1], alpha=0.10, ax=axes['b'])
    sns.scatterplot(x=X_res.iloc[y_res.values == 1, 0], y=X_res.iloc[y_res.values == 1, 1], alpha=0.10, ax=axes['b'])
    sns.countplot(y=y, ax=axes['c'])
    sns.countplot(y=y_res, ax=axes['d'])
    # Set titles and show plot
    plt.suptitle(method + ' Outcomes')
    axes['a'].title.set_text('Original Data')
    axes['b'].title.set_text(method + ' Data')
    axes['c'].title.set_text('Original Class Proportions')
    axes['d'].title.set_text(method + ' Class Proportions')
    plt.tight_layout()
    plt.show()

## Load in your data
The cell below is used to import the data. The example dataset used here is for fraud data where there is a large imbalance between fraudulent and non-fraudulent cases.

👇&nbsp;&nbsp;_To use your own data, you will need to:_
- _Upload a file and update the path provided to `pd.read_csv()`._
- _Alternatively, if you have data in a database, you can add a SQL cell and connect to a custom integration._
- _Your data will need to contain numeric data for the target and feature variables. It will also need to already be pre-processed (i.e., clean and with pre-processing steps already completed)._

In [ ]:
# Import the data (add your own file here)
df = pd.read_csv("data/fraud.csv")

# Preview the data
df

## How imbalanced is my data?
The first step is to prepare the data and investigate the extent of the imbalance. This includes splitting your data into training and testing subsets and then visualizing the imbalance in your target variable.

👇&nbsp;&nbsp;_In this cell, you will need to:_
- _Set the name of the target variable._
- _Define which variables are your features._

In [ ]:
# Set target and feature variables (add your own here)
target = "is_fraud"
features = ["lat", "long", "amt", "city_pop", "merch_lat", "merch_long"]

# Separate the target from the features
y = df[target]
X = df[features]

# Split the data into train and testing subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Plot the counts of the 
sns.countplot(y=y_train) 
plt.title("Imbalance Between Classes", size=16)
plt.show()

Based on the size and type of your data, you will next want to choose a method to correct the imbalance between classes. You can navigate to the following sections to find the code to:
- Perform [random under-sampling](#rus) to reduce the quantity of the majority class.
- Perform [random over-sampling](#ros) to increase the quantity of the minority class.
- Perform [SMOTE](#smote) over-sampling to increase the quantity of the minority class.

<a id='rus'></a>
## Randomly sampling our majority class to match the size of the minority
Random under-sampling is a technique that performs random draws from the majority class to match the minority class. As a result, you risk throwing away a lot of data. You will probably only want to use random under-sampling when:
- You have a lot of data that you can afford to discard.
- You are aiming for computational efficiency.
The following code uses [RandomUnderSampler()](https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.RandomUnderSampler.html) to under-sample the majority class to match the minority class.

💡&nbsp;&nbsp;_For `RandomUnderSampler()`, and the other sampling classes introduced in this template, there is an optional `sampling_strategy` parameter. Passing a float between 0 and 1 as an argument will adjust the ratio of samples in the minority class._

In [ ]:
# Import RandomUnderSampler
from imblearn.under_sampling import RandomUnderSampler

# Resample the data
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)

# View the resampled labels
plot_resample(X_train, y_train, X_res, y_res, "Random Under Sampling")

💡&nbsp;&nbsp;_Points in the upper plots are slightly transparent, which means that highly opaque points indicate a greater concentration of values. Notice how there is greater transparency in the random undersampled data, indicating fewer data points (confirmed by the lower right barplot) and a greater balance of the green and markers, indicating an equal quantity between classes._

<a id='ros'></a>
## Randomly sampling to increase the size of our minority class
If you wish to avoid the loss of data that comes with random under-sampling, you can choose to use random over-sampling. This technique performs random draws with replacement on the minority class to match the majority class. This is a simple technique but has the drawback that your model will be trained on a large number of duplicates which may harm model performance by introducing issues such as overfitting.

The following code uses [RandomOverSampler()](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.RandomOverSampler.html) to over-sample the minority class to match the majority class.

In [ ]:
# Import RandomOverSampler
from imblearn.over_sampling import RandomOverSampler

# Resample the data
ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X_train, y_train)

# View the resampled labels
plot_resample(X_train, y_train, X_res, y_res, "Random Over Sampling")

💡&nbsp;&nbsp;_Notice here that unlike the random under sampling, there is a similar level of opacity between the two plots. This indicates that the quantity of data was preserved after resampling. There is also now a similar quantity of orange and blue points, indicating an equality between the two classes (confirmed by the lower right bar plot)._

<a id='smote'></a>
## Creating synthetic samples to rebalance our data
The final technique this template will cover is SMOTE or Synthetic Minority Over-sampling Technique. SMOTE functions works like random over-sampling, but instead of exact duplicates, it creates new synthetic observations. Although this reduces the risk of overfitting, it can have some potential downsides:
- SMOTE does not work as well with high-dimensional data.
- SMOTE can introduce noise into the data because of how it generates synthetic examples.

The following code uses [SMOTE()](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html) to use examples from the minority class to generate new synthetic data points to match the majority class.

💡&nbsp;&nbsp;_If your dataset contains categorical features, such as one-hot encoded variables, you will need to use [SMOTENC](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTENC.html) instead of SMOTE. Visit the documentation to learn more._

In [ ]:
# Import SMOTE
from imblearn.over_sampling import SMOTE

# Resample the data
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

# View the resampled labels
plot_resample(X_train, y_train, X_res, y_res, "SMOTE")

💡&nbsp;&nbsp;_You may notice two features of the plot above:_
- _Lines of points in the plot of the SMOTE data. This is caused by the way in which the new samples are generated through interpolation. You can read more about how SMOTE works [here](https://imbalanced-learn.org/stable/over_sampling.html#smote-adasyn)._
- _An abundance of the minority class. This is because in the example data there are many duplicate values. Via interpolation, the minority class will have fewer duplicates, and will therefore cover more of the plot._

## Further reading
There are a number of ways to proceed, including:
- Applied use cases where you might encounter class imbalances, such as fraud detection. DataCamp's [Fraud Detection in Python](https://app.datacamp.com/learn/courses/fraud-detection-in-python) covers this topic in greater depth.
- More advanced forms of sampling methods, including combinations such as [SMOTEEN](https://imbalanced-learn.org/stable/references/generated/imblearn.combine.SMOTEENN.html#imblearn.combine.SMOTEENN) and [SMOTETomek](https://imbalanced-learn.org/stable/references/generated/imblearn.combine.SMOTETomek.html#imblearn.combine.SMOTETomek).
- With a balanced dataset, you may want to check out [workspace templates](https://app.datacamp.com/workspace/templates?selectedLabels=%5B%22classification%22%5D) focused on classification problems.